In [1]:
from datasetsforecast.long_horizon import LongHorizon

Y_df, X_df, S_df = LongHorizon.load(directory='data', group='ETTh1')

In [2]:
import pandas as pd

Y_df["ds"] = pd.to_datetime(Y_df["ds"])
X_df["ds"] = pd.to_datetime(X_df["ds"])

In [3]:
import logging

from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS, RNN, TFT

logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)

In [4]:
horizon = 12

# Try different hyperparmeters to improve accuracy.
models = [
      LSTM(
            input_size=2 * horizon,
            h=horizon,                    # Forecast horizon
            max_steps=500,                # Number of steps to train
            scaler_type='standard',       # Type of scaler to normalize data
            encoder_hidden_size=64,       # Defines the size of the hidden state of the LSTM
            decoder_hidden_size=64,       # Defines the number of hidden units of each layer of the MLP decoder
      ),     
      NHITS(
            h=horizon,                   # Forecast horizon
            input_size=2 * horizon,      # Length of input sequence
            max_steps=100,               # Number of steps to train
            n_freq_downsample=[2, 1, 1]  # Downsampling factors for each stack output
      ),
      TFT(
            h=horizon,                   # Forecast horizon
            input_size=2 * horizon,      # Length of input sequence
            max_steps=100,               # Number of steps to train
      )
]
nf = NeuralForecast(models=models, freq='h')
nf.fit(df=Y_df)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [5]:
Y_hat_df = nf.predict()

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [6]:
Y_hat_df = Y_hat_df
Y_hat_df.head()

,unique_id,ds,LSTM,NHITS,TFT
0,OT,2018-02-21 00:00:00,-1.571183,-1.603926,-1.542601
1,OT,2018-02-21 01:00:00,-1.549101,-1.591675,-1.520578
2,OT,2018-02-21 02:00:00,-1.542599,-1.587134,-1.511382
3,OT,2018-02-21 03:00:00,-1.543403,-1.587013,-1.513252
4,OT,2018-02-21 04:00:00,-1.503202,-1.579409,-1.527575


In [7]:
from utilsforecast.plotting import plot_series

In [8]:
plot_series(Y_df, Y_hat_df, engine="plotly-resampler")